In [19]:
import os
import pandas as pd
import base64
import mimetypes
from openai import OpenAI
from dotenv import load_dotenv
import json
from datasets import Dataset, ClassLabel, Sequence, DatasetDict
from tqdm import tqdm
from pathlib import Path
from huggingface_hub import login
import pickle

# Provide necessary API keys and tokens

Create a `.env` file and add your OpenAI API key and Hugging Face token like so:
```plaintext
OPENAI_API_KEY=your_openai_api_key_here
HF_TOKEN=your_hugging_face_token_here
```
Your will need to add a Hugging Face token with write privileges to push the created dataset to Hugging Face Hub.

In [2]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY'),
)

login(token=os.environ.get('HF_TOKEN'))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Auxiliary functions

In [18]:
def text_only_labeling(text, tags):
    prompt = f"""
    You are an NER tagging assistant. Your task is to label all entities in the text based on the tags provided.
    Here are the tags: {', '.join(tags)}
    Use {tags[0]} whenever an entity does not match any other tag. It is imperative that you label everything in the text. Don't make the entities too short.

    For each entity, return the entity and its tag.

    Input text: "{text}"

    Respond with the entities in this JSON format:
    [
        {{ "entity": str, "tag": str }},
        ...
    ]
    """
    
    response = client.chat.completions.create(
        messages=[{'role': 'user', 'content': prompt}],
        model='gpt-4o',
        temperature=0,
        response_format={'type': 'json_object'},
    )

    return response.choices[0].message.content

def parse_response(response):
    try:
        parsed_data = json.loads(response)
    except json.JSONDecodeError as e:
        raise ValueError(f'Error parsing JSON: {e}, in response: {response}')
        
    tokens = []
    tags = []

    if type(parsed_data) is list:
        entities = parsed_data
    elif type(parsed_data) is dict:
        if 'entities' in parsed_data.keys():
            entities = parsed_data['entities']
        else:
            entities = [parsed_data]
        
    for entity in entities:
        token = entity.get('entity')
        tag = entity.get('tag')
        if token is not None and tag is not None:
            tokens.append(token)
            tags.append(tag)
        else:
            raise ValueError(f'Unexpected response format: {response}')

    return tokens, tags

def chunk_string_by_words(text, chunk_size):
    words = text.split()
    chunks = [words[i:i + chunk_size] for i in range(0, len(words), chunk_size)]
    chunked_text = [' '.join(chunk) for chunk in chunks]
    
    return chunked_text

def generate_data_from_csv(path, ner_tags):
    print(f'generating data using {path}')
    
    tokens_list = []
    tags_list = []
    
    df = pd.read_csv(path)
    df.columns = df.columns.str.replace(' ', '_').str.lower()

    if 'text' not in df.columns:
        raise ValueError('no text column in csv')
    
    grouping_columns = ['application_name', 'seen_timestamp']

    for col_name in grouping_columns:
        if col_name not in df.columns:
            raise ValueError(f'no {col_name} column in csv')
    
    grouped_dfs = [group for _, group in df.groupby(grouping_columns)]
    view_texts_coalesced = [' '.join(df['text'].dropna()) for df in grouped_dfs]
    view_texts_filtered = [s for s in view_texts_coalesced if s != '']
    view_texts_chunked = []

    for text in view_texts_filtered:
        view_texts_chunked += chunk_string_by_words(text, 64)
    
    for text in tqdm(view_texts_chunked):
        try:
            response = text_only_labeling(text, ner_tags)
            tokens, tags = parse_response(response)
            tokens_list.append(tokens)
            tags_list.append(tags)
        except Exception as e:
            print(e)

    return tokens_list, tags_list

def create_dataset(tokens_list, tags_list, ner_tags):
    dataset = Dataset.from_dict({'tokens': tokens_list, 'ner_tags': tags_list})
    dataset = dataset.cast_column('ner_tags', Sequence(feature=ner_tags))
    dataset = dataset.add_column('id', [i for i in range(len(dataset))])
    
    trainvalid_test = dataset.train_test_split(test_size=0.2)
    train_valid = trainvalid_test['train'].train_test_split(test_size=0.2)
    split_dataset = DatasetDict({
        'train': train_valid['train'],
        'test': trainvalid_test['test'],
        'validation': train_valid['test']}
    )
    
    return split_dataset

def checkpoint_name(path):
    return f'{os.path.join('checkpoints', Path(path).stem)}.pkl'

# Generate data

This part of the script will fetch all csv files from a `data` directory and attempt to generate labeled data with them.

In [6]:
data_path = Path('data')
ner_tags = ClassLabel(names=['N/A', 'PRICE', 'PRODUCT', 'DISCOUNT_PERCENTAGE', 'QUANTITY', 'DATE'])

for csv_file in data_path.glob('*.csv'):
    if os.path.exists(checkpoint_name(csv_file)):
        print(f'skipping {csv_file}')
        continue
        
    try:
        tokens_list, tags_list = generate_data_from_csv(csv_file, ner_tags.names)
        with open(checkpoint_name(csv_file), 'wb') as f:
            print(f'saving checkpoint {checkpoint_name(csv_file)}')
            pickle.dump((tokens_list, tags_list), f)
        
    except ValueError as e:
        print(e)

generating data using data\18789327023.csv



00%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:27<00:00,  3.97s/it]

saving checkpoint checkpoints\18789327023.pkl
generating data using data\18929485529.csv



00%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:21<00:00,  3.63s/it]

saving checkpoint checkpoints\18929485529.pkl
generating data using data\test_data_2024_03_07_dm_content_generic_2024-12-05T10_09_32.502568365+01_00.csv


...

saving checkpoint checkpoints\test_data_2024_03_07_rewe_content_generic_2024-12-05T10_30_59.948177782+01_00.pkl
generating data using data\test_data_2024_03_07_rossmann_content_generic_2024-12-05T10_24_07.981399375+01_00.csv



00%|████████████████████████████████████████████████████████████████████████████████| 141/141 [08:31<00:00,  3.63s/it]

saving checkpoint checkpoints\test_data_2024_03_07_rossmann_content_generic_2024-12-05T10_24_07.981399375+01_00.pkl
generating data using data\test_data_2024_11_25_lidl_plus_content_generic_2024-12-05T07_39_49.726955559+01_00.csv


100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [08:21<00:00,  4.60s/it]

saving checkpoint checkpoints\test_data_2024_11_25_lidl_plus_content_generic_2024-12-05T07_39_49.726955559+01_00.pkl


# Create dataset

In [20]:
tokens_list = []
tags_list = []

for pkl_file in Path('checkpoints').glob('*.pkl'):
    with open(pkl_file, 'rb') as f:
        tokens, tags = pickle.load(f)

    tokens_list += tokens
    tags_list += tags

dataset = create_dataset(tokens_list, tags_list, ner_tags)

Casting the dataset:   0%|          | 0/605 [00:00<?, ? examples/s]

# Save dataset

In [21]:
dataset.push_to_hub("murmuras_labeled_data_from_csv", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kdybek/murmuras_labeled_data_from_csv/commit/aeff8795fae073199213983ea517fc2a2eed5fe2', commit_message='Upload dataset', commit_description='', oid='aeff8795fae073199213983ea517fc2a2eed5fe2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kdybek/murmuras_labeled_data_from_csv', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kdybek/murmuras_labeled_data_from_csv'), pr_revision=None, pr_num=None)